In [3]:
    import requests
    import pandas as pd
    from datetime import date
    import numpy as np

    # URL template (corrected)
    url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-03-11&to=2025-06-15"

    # Instrument tokens
    tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

    # Interval mapping
    KI = {
        "3m": "3minute",
        "5m": "5minute",
        "15m": "15minute",
        "60m": "60minute",
        "day": "day",
    }

    # Function to get candle data
    def getCandles(symbol, fromDate, toDate, timeframe):
        headers = {
            "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
            # Replace this with your actual token securely
        }
        curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
        print("Requesting:", curUrl)

        try:
            session = requests.session()
            response = session.get(curUrl, headers=headers)
            r = response.json()
        except Exception as error:
            print("API request failed:", error)
            return pd.DataFrame()

        try:
            history = r["data"]["candles"]
            history_df = pd.DataFrame(history)
            history_df = history_df.rename(
                columns={
                    0: "Datetime",
                    1: "open",
                    2: "high",
                    3: "low",
                    4: "close",
                    5: "volume",
                }
            )
            history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
            history_df.set_index("Datetime", inplace=True)
            return history_df
        except Exception as error:
            print("Data processing failed:", error)
            return pd.DataFrame()


    # Shooting Star detection
    def shooting_star(ohlc_df):
        df = ohlc_df.copy()
        body = abs(df["close"] - df["open"])
        candle_range = df["high"] - df["low"]
        upper_shadow = df["high"] - df[["close", "open"]].max(axis=1)
        lower_shadow = df[["close", "open"]].min(axis=1) - df["low"]

        df["sstar"] = (
            (candle_range > 3 * body)
            & (upper_shadow / (0.001 + candle_range) > 0.6)
            & (lower_shadow / (0.001 + candle_range) < 0.2)
        )

        return df



    print("Start")
    symbol_data = getCandles("NIFTY", "2025-03-20", "2025-03-25", KI["15m"])


    if not symbol_data.empty:
        ss_df = shooting_star(symbol_data)
        print(ss_df[ss_df["sstar"]])
    else:
        print("No OHLC data available.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-03-11&to=2025-06-15
                               open      high       low     close  volume  6  \
Datetime                                                                       
2025-03-11 10:55:00+05:30  22443.30  22468.95  22439.40  22450.70       0  0   
2025-03-11 11:25:00+05:30  22428.70  22444.80  22421.85  22422.60       0  0   
2025-03-11 13:25:00+05:30  22465.15  22475.50  22461.75  22462.40       0  0   
2025-03-11 14:45:00+05:30  22487.65  22507.65  22486.15  22492.90       0  0   
2025-03-12 09:15:00+05:30  22536.35  22577.40  22531.40  22534.70       0  0   
...                             ...       ...       ...       ...     ... ..   
2025-06-13 10:45:00+05:30  24671.85  24688.05  24668.25  24675.60       0  0   
2025-06-13 11:45:00+05:30  24678.70  24687.30  24673.15  24675.65       0  0   
2025-06-13 12:05:00+05:30  24690.35  24702.10  24685.05  24686.40